In [7]:
import os
import datetime
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [8]:
cwd = os.getcwd()
output_dir = os.path.join(cwd, "../../data")
df = pd.read_csv(os.path.join(output_dir, "combined_figlet.csv"))
df_test = pd.read_csv(os.path.join(output_dir, "combined_figlettest.csv"))
#df_test = pd.read_csv(os.path.join(output_dir, "figlet1599056546.006096_1599063766.431225.csv"))
df = df.drop(df[df.responsetime > 2].index)
df.pop('Time')
df_test = df_test.drop(df_test[df_test.responsetime > 2].index)
df_test.pop('Time')
df.head()

,cpu,diskio,diskioRead,diskioWritten,mem,networkReceived,networkTransmitted,replica,requests,responsetime,totalcpu,totalcpuUtilization,totalmemory,totalmemoryUtilization
20,0.574245,0.016356,0.0,101580.800000,0.032037,25956.422222,45677.777778,5.0,2939.0,0.104074,24.0,3.805556,1.011896e+11,5.741587
21,0.566708,0.015378,0.0,96847.644444,0.035930,26051.266667,45342.311111,5.0,3090.0,0.102654,24.0,3.805556,1.011896e+11,5.741587
22,0.668652,0.016267,0.0,99851.377778,0.040543,28178.400000,49011.688889,7.0,3229.0,0.099415,24.0,3.788889,1.011896e+11,5.763427
23,0.668899,0.018222,0.0,107406.222222,0.040543,30325.511111,52805.200000,7.0,3377.0,0.106906,24.0,3.788889,1.011896e+11,5.763427
24,0.622481,0.017067,0.0,93115.733333,0.044674,31841.888889,51576.533333,7.0,3522.0,0.107957,24.0,4.230556,1.011896e+11,5.792171


In [9]:
df_t = pd.DataFrame()
df_ttest = pd.DataFrame()

In [10]:
df_t['total_cpu_util'] = (df['totalcpuUtilization']*df['totalcpu'])/100
df_t['total_mem_util'] = (df['totalmemoryUtilization']*df['totalmemory'])*1e-9
df_t['responsetime'] = df['responsetime']
df_t['requests'] = df['requests']
df_t['replica'] = df['replica']
df_ttest['total_cpu_util'] = (df_test['totalcpuUtilization']*df_test['totalcpu'])/100
df_ttest['total_mem_util'] = (df_test['totalmemoryUtilization']*df_test['totalmemory'])*1e-9
df_ttest['responsetime'] = df_test['responsetime']
df_ttest['requests'] = df_test['requests']
df_ttest['replica'] = df_test['replica']
df_t.head()

,total_cpu_util,total_mem_util,responsetime,requests,replica
20,0.913333,580.988928,0.104074,2939.0,5.0
21,0.913333,580.988928,0.102654,3090.0,5.0
22,0.909333,583.198925,0.099415,3229.0,7.0
23,0.909333,583.198925,0.106906,3377.0,7.0
24,1.015333,586.107494,0.107957,3522.0,7.0


In [14]:
n = len(df_t)
train_df = df_t[0:int(n*0.8)]
eval_df = df_t[int(n*0.8):]
test_df = df_ttest[:]
x_train = train_df[[col for col in train_df.columns if col != 'requests']]
y_train = train_df['requests']
x_train = preprocessing.normalize(x_train, norm='max')
x_val = eval_df[[col for col in eval_df.columns if col != 'requests']]
y_val = eval_df['requests']
x_val = preprocessing.normalize(x_val, norm='max')
x_test = test_df[[col for col in eval_df.columns if col != 'requests']]
y_test = test_df['requests']
x_test = preprocessing.normalize(x_test, norm='max')

In [15]:
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [16]:
from keras.layers import Dropout

# Initialising the RNN
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units = 1))

In [17]:
#compile and fit the model on 30 epochs
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 30, batch_size = 50)

Epoch 1/30
49/49 [==============================] - 1s 14ms/step - loss: 1141052544.0000
Epoch 2/30
49/49 [==============================] - 1s 13ms/step - loss: 1140513408.0000
Epoch 3/30
49/49 [==============================] - 1s 13ms/step - loss: 1140326272.0000
Epoch 4/30
49/49 [==============================] - 1s 13ms/step - loss: 1140161792.0000
Epoch 5/30
49/49 [==============================] - 1s 14ms/step - loss: 1140006400.0000
Epoch 6/30
49/49 [==============================] - 1s 13ms/step - loss: 1139852800.0000
Epoch 7/30
49/49 [==============================] - 1s 13ms/step - loss: 1139704320.0000
Epoch 8/30
49/49 [==============================] - 1s 13ms/step - loss: 1139547904.0000
Epoch 9/30
49/49 [==============================] - 1s 15ms/step - loss: 1139403008.0000
Epoch 10/30
49/49 [==============================] - 1s 15ms/step - loss: 1139251200.0000
Epoch 11/30
49/49 [==============================] - 1s 11ms/step - loss: 1139105792.0000
Epoch 12/30
49/49 [

In [18]:
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [20]:
#check predicted values
predictions = model.predict(x_test) 
print(predictions)

[[86.31051 ]
 [86.3105  ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31044 ]
 [86.31044 ]
 [86.31044 ]
 [86.31044 ]
 [86.310455]
 [86.310455]
 [86.31044 ]
 [86.31044 ]
 [86.31044 ]
 [86.31043 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.310455]
 [86.310455]
 [86.31045 ]
 [86.31044 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31044 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.31044 ]
 [86.31044 ]
 [86.31045 ]
 [86.31045 ]
 [86.31045 ]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.31045 ]
 [86.31045 ]
 [86.31044 ]
 [86.31044 ]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.31046 ]
 [86.31046 ]
 [86.31046 ]
 [86.31046 ]
 [86.31046 ]
 [86.310455]
 [86.31045 ]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]
 [86.310455]